# Features as Tools 

This tutorial guides you through creating an LLM generated restaurant recommendation function by using Features as Tools.

This is an example of how an LLM can make use of feature views as data retrieval tools. At development time, you provide the set of feature views that the LLM agent is allowed to call. The LLM uses the feature view's description field to determine what tools are relevant for answering the user question. It  makes the corresponing feature retrieval call(s) using the Tecton entity keys provided in the context of the call.


## Install Packages

In [ ]:
!pip install 'tecton[rift]==0.10.0b32' gcsfs s3fs --force-reinstall
!pip install "$HOME/Downloads/tecton_gen_ai-0.0.4-py3-none-any.whl" --force-reinstall

!pip install openai
!pip install langchain-openai
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install llama-index
!pip install llama-index-llms-openai

# Features as Tools

In the following cell you'll create a Tecton Agent with a simple prompt and two different feature views for the LLM as tools.
The system prompt in this example, instructs the LLM to address the user by name and to only suggest restaurants that in near the location. In order to address the use by name the LLM will need to use a tools.
The location is configured to be provided as part of the context in the request by using a RequestSource object. 

In [5]:
from tecton_gen_ai.agent import AgentService, AgentClient
from tecton_gen_ai import prompt
from tecton_gen_ai.core_utils import make_request_source

def restaurant_recommender_agent( user_info_fv, recent_visit_fv):    

    location_request = make_request_source(location = str)
    
    @prompt(sources=[ location_request])
    def sys_prompt(location_request ):
        location = location_request["location"]
        return f"""
        Address the user by name.
        You are a consierge service that recommends restaurants.
        Respond to the user's question. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Only suggest restaurants that are in or near {location}. 
        """
            
    return AgentService(
        name="restaurant_recommender",
        prompts=[sys_prompt],
        tools=[ user_info_fv, recent_visit_fv],  # feature views used as tools
    )

The `sys_prompt` instructions above require the LLM to "address the user by name" which requires the use of a tool.
 
In the next cell we create a couple of mock feature views that provide basic user information and the list of their recently visited restaurants respectively.

In practice, these feature views would be implemented as stream feature views that would provide the latest user information and recent visits within seconds of the corresponding user activity.

In [6]:
import pandas as pd
from tecton import RequestSource
from tecton.types import Field, String


from tecton_utils.testing import mock_batch_feature_view


# mock user info data
user_info = pd.DataFrame(
        [
            {"user_id": "user1", "name": "Jim",  "age": 30, "food_preference": "American"},
            {"user_id": "user2", "name": "John", "age": 40, "food_preference": "Italian"},
            {"user_id": "user3", "name": "Jane", "age": 50, "food_preference": "Chinese"},
        ]
    )
user_info_fv = mock_batch_feature_view( "user_info_fv", user_info, entity_keys=["user_id"], description="User's basic information.")

# mock user's recent visits 
recent_eats = pd.DataFrame(
        [
            {"user_id": "user1", "last_3_visits":str(["Mama Ricotta's", "The Capital Grille", "Firebirds Wood Fired Grill"])},
            {"user_id": "user2", "last_3_visits":str(["Mama Ricotta's", "Villa Antonio", "Viva Chicken"])},
            {"user_id": "user3", "last_3_visits":str(["Wan Fu", "Wan Fu Quality Chinese Cuisine", "Ru San's"])},
        ]
    )
recent_eats_fv = mock_batch_feature_view( "recent_eats_fv", recent_eats, entity_keys=["user_id"], description="User's recent restaurant visits.")



The feature views entity key in both cases is `user_id`, the key will need to be provided in the context when invoking the LLM agent. 

In the following cell, you create a Tecton AgentClient with uses the mock feature views defined above.

In [7]:
# create the Tecton Agent
recommender_agent = restaurant_recommender_agent( user_info_fv, recent_eats_fv )

# create a client to invoke the agent
client = AgentClient.from_local( recommender_agent )

## Put it all together

The Tecton AgentClient created above can be used to create a LangChain or LlamaIndex runnable agent.

In the cell below you will instantiate an LLM model using OpenAI's `gpt-4o-mini` model and create a LangChain agent that is ready to use the tools we've provided above.

Obtain an [OpenAI API key](https://platform.openai.com/api-keys) and replace "your-openai-key" in the following cell.

In [9]:
import openai as oa
import os
from langchain_openai import ChatOpenAI
from llama_index.llms.openai import OpenAI

from langchain_core.output_parsers import StrOutputParser


# replace with your key
os.environ["OPENAI_API_KEY"] = "your-openai-key"

# instantiate LLM model for both LangChain & LlamaIndex showing same integration with both
langchain_llm = ChatOpenAI(model="gpt-4o-mini")
llamaindex_llm = OpenAI(model="gpt-4o-mini")

# create invokable agents for LangChain & LlamaIndex 
langchain_agent = client.make_agent(langchain_llm, system_prompt="sys_prompt")
llamaindex_agent = client.make_agent(llamaindex_llm, system_prompt="sys_prompt")


### Testing with different context

In the following cells you'll provide different context and invoke the LLM agent to test how features as tools provide user specifc personalization, you'll notice that we can either use the langchain or llamaindex agents created above with the same results. 

Notice that the function name and signature vary between the two because they langchain and llamaindex invoke methods are different and have different input/ouput signatures.

The first test uses the `langchain_agent`, it's input and output are a dictionaries with "input" and "output" keys:

In [10]:
print("Context - user 1 in Charlotte \n\nResponse:")
with client.set_context({"user_id": "user1", "location":"Ballantyne, Charlotte, NC"}):
    print(langchain_agent.invoke({"input":"what do you know about me and what would you recommend"})["output"])

Context - user 1 in Charlotte 

Response:
Hello Jim! Based on your recent visits, it seems you enjoy American cuisine. 

I recommend trying **The Palm** in Ballantyne. It's a classic American steakhouse known for its exceptional service and delicious food. 

**Menu suggestions:**
- **Filet Mignon** - Tender and cooked to perfection.
- **Lobster Mac & Cheese** - A rich and creamy side that's a crowd favorite.
- **Chocolate Cake** - A must-try for dessert; it's decadent and satisfying.

Enjoy your dining experience!


## Test it out

In the following cells you can see how the response changes based on the `user_id` and the `location` provided resulting in a personalized response for each user and based on their current location.

The following test uses the `llamaindex_agent` with its `chat` method that receiving a query string as input and returns output as a string as well:

In [18]:
print( "Context - user 2 in Charlotte \n\nResponse:")
with client.set_context({"user_id": "user2", "location":"Ballantyne, Charlotte, NC"}):
    print(llamaindex_agent.chat("I want to try a place tonight that I've never been to that also fits my preference"))


Context - user 2 in Charlotte 

Response:
Hi John! Another great option for Italian cuisine that you haven't tried yet is **Maggiano's Little Italy** in the nearby area. 

They have a wonderful selection of classic Italian dishes. You might want to try their **Fettuccine Alfredo** or the **Lasagna**. For an appetizer, the **Stuffed Mushrooms** are highly recommended, and for dessert, their **Tiramisu** is a must-try!

Enjoy your evening out!


In [ ]:
print( "Context - user 3 in Charlotte \n\nResponse:")
with client.set_context({"user_id": "user3", "location":"Charlotte, NC"}):
    print(llamaindex_agent.chat("Recommend one of my regular spots for dinner"))


Context - user 3 in Charlotte 

Response:
I recommend going back to **Mama Ricotta's**! It's a wonderful Italian restaurant where you can enjoy their delicious **Lasagna** or the **Pasta Carbonara**. 

For an appetizer, the **Bruschetta** is a great choice, and if you have room for dessert, their **Tiramisu** is a must-try. Enjoy your meal!


In [22]:
print( "Context - user 3 in Charlotte \n\nResponse:")
with client.set_context({"user_id": "user3", "location":"Charlotte, NC"}):
    print(langchain_agent.chat("Recommend one of my regular spots for dinner"))


Context - user 3 in Charlotte 

Response:
I recommend going back to **Mama Ricotta's**! It's a wonderful Italian restaurant where you can enjoy their delicious **Lasagna** or the **Pasta Carbonara**. 

For an appetizer, the **Bruschetta** is a great choice, and if you have room for dessert, their **Tiramisu** is a must-try. Enjoy your meal!


# Conclusion

Tecton delivers real-time, streaming and batch feature pipelines in production. Enhancing an LLMs context through features as tools, brings the power of fresh context to generative AI applications. By using features as tools for the LLM, you give the LLM the ability to retrieve relevant information only when the user question needs requires it.